# Energy Price

This application retrieves the price information provided by the Entsoe transparency portal, 
starting from the requested date up to today's date. The values are transformed from MWh to KWh.

#### Importing Libraries

In [1]:
import pandas as pd
import requests
import csv
from datetime import timedelta, datetime, date

#### Define the url

In [2]:
# URL
start_url = 'https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime='
end_url = '+00:00|CET|DAY&biddingZone.values=CTY|10YNL----------L!BZN|10YNL----------L&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)'

#### Read the csv file

In [3]:
df = pd.read_csv('EntsoeEnergyPrice.csv', parse_dates=['Date'], dayfirst=True)

#### Define the Date

In [4]:
# The earliest date in the dataset
min_date = pd.to_datetime(df['Date'].min(), format='%Y-%m-%d').date()

# The latest date in dataset
max_date = pd.to_datetime(df['Date'].max(), format='%Y/%m/%d').date()

# Today date
today = datetime.today().date()

print('Earliest date found in the dataset:', min_date)
print('Latest date found in the dataset:', max_date)
print('Today\'s date:', today)

Earliest date found in the dataset: 2021-07-12
Latest date found in the dataset: 2023-05-08
Today's date: 2023-05-11


In [5]:
# Create list of the days that are missing
datesL = [] # Lastest Dates
datesE = [] # Earlist Dates

# Days of difference between today and lastest date
dif_days = (today - max_date).days

# Check if the dataset is updated
if dif_days != 0:
    print('The difference between the latest date and today is:', dif_days, 'days')
else:
    print('The dataset is updated!')

# Updated the dates of interesting
# Lastest Dates
if dif_days != 0:
    for i in range(dif_days):
        max_date += timedelta(days=1)
        datesL.append(max_date.strftime("%d-%m-%Y"))
        
# Earlist Dates
for i in range(30):
    min_date = min_date - timedelta(days=1)
    datesE.append(min_date.strftime("%d-%m-%Y"))

    
print(min_date)
print(max_date)    
    
print(datesL)
print(datesE)

The difference between the latest date and today is: 3 days
2021-06-12
2023-05-11
['09-05-2023', '10-05-2023', '11-05-2023']
['11-07-2021', '10-07-2021', '09-07-2021', '08-07-2021', '07-07-2021', '06-07-2021', '05-07-2021', '04-07-2021', '03-07-2021', '02-07-2021', '01-07-2021', '30-06-2021', '29-06-2021', '28-06-2021', '27-06-2021', '26-06-2021', '25-06-2021', '24-06-2021', '23-06-2021', '22-06-2021', '21-06-2021', '20-06-2021', '19-06-2021', '18-06-2021', '17-06-2021', '16-06-2021', '15-06-2021', '14-06-2021', '13-06-2021', '12-06-2021']


#### Get the Price information based on the dates

In [6]:
# Create a temporary DataFrame to store the daily price 
df_comb = pd.DataFrame() # Temporary DataFrame combine the prices with the original
df_tempE = pd.DataFrame() # Temporary DataFrame for prices (earliest dates)
df_tempL = pd.DataFrame() # Temporary DataFrame for prices (latest dates)

# Updating the dataset with latest dates
if dif_days != 0:
    for actual_date in datesL:
        # Create a URL with the new date
        url = start_url + actual_date.replace("-", ".") + end_url

        # Copy the information from the URL into a temporary DataFrame
        df_tempL = pd.read_html(url)[0]

        # Add a Date column to the temporary DataFrame
        df_tempL['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

        # Append the information from the temporary DataFrame to the combined DataFrame
        
        df_comb = pd.concat([df_comb, df_tempL], ignore_index=True)

        #     url = start_url + actual_date.replace("-", ".") + end_url

# Copy the information from the URL into a temporary DataFrame
df_tempE = pd.read_html(url)[0]

# Add a Date column to the temporary DataFrame
df_tempE['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Append the information from the temporary DataFrame to the combined DataFrame
df_comb = pd.concat([df_comb, df_tempE], ignore_index=True)

#### Cleaning the data

In [7]:
#Drop the first level of columns
df_comb.columns = df_comb.columns.droplevel()

#Rename the Columns
df_comb = df_comb.rename(columns={
    "index": "Index",
    "MTU": "start-end [time]",
    "Day-ahead Price": "Import Grid (EUR/kWh)",
    "": "Date"
})

#Split the hours in start e end
df_comb[['Hour', 'End']] = df_comb['start-end [time]'].str.split('-', expand=True)

#Remove blank spaces
df_comb['Hour'] = df_comb['Hour'].str.strip()
df_comb['End'] = df_comb['End'].str.strip()

#Remove the original column
df_comb.drop('start-end [time]', axis=1, inplace=True)
df_comb.drop('End', axis=1, inplace=True)

#Creating a new column Price and converting the MWh to KWh
df_comb['Import Grid (EUR/kWh)'] = df_comb['[EUR / MWh]'] / 1000
df_comb = df_comb.drop('[EUR / MWh]', axis=1)


In [8]:
df_comb['Export Grid (EUR/kWh)'] = df_comb['Import Grid (EUR/kWh)']

# Substituir valores maiores do que zero por 'ZERO' na coluna Export Grid (EUR/kWh)
df_comb.loc[df_comb['Import Grid (EUR/kWh)'] < 0, 'Import Grid (EUR/kWh)'] = 0

# Substituir valores menores ou iguais a zero por zero na coluna Import Grid (EUR/kWh)
df_comb.loc[df_comb['Export Grid (EUR/kWh)'] > 0, 'Export Grid (EUR/kWh)'] = 0
df_comb.loc[df_comb['Export Grid (EUR/kWh)'] < 0, 'Export Grid (EUR/kWh)'] = df_comb['Export Grid (EUR/kWh)']*(-1)

#### Salving the date in a CSV file

In [9]:
# Save in a csv file
df.to_csv('EntsoeEnergyPrice.csv', index=False)